In [2]:
# Import libraries
import pandas as pd
import numpy as np
import os

from dvc.api import make_checkpoint

# Display full outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings("ignore")

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/project/spark-3.2.1-bin-hadoop3.2"

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("PySpark App") \
    .config("spark.jars", "postgresql-42.3.2.jar") \
    .getOrCreate()

### Prepare Subsets for Schema

In [57]:
# Get all the tables based on the schema
# Film information tables
film_info = df_film[['title','year','genre', 'duration','rating']]
make_checkpoint()
crew_cast = df_film[['title','director','cast', 'distributor']]
make_checkpoint()
income = df_film[['title','domestic_k','international_k', 'worldwide_k', 'dom_pct', 'int_pct']]
make_checkpoint()
ranking = df_film[['title','place','duration_rank', 'income_rank', 'rank_diff']]
make_checkpoint()

In [58]:
film_info

,title,year,genre,duration,rating
0,Harry Potter and the Deathly Hallows: Part 2,2011,Adventure,130,8.0748
1,Mr Smith Goes to Washington,1939,Comedy,129,8.0725
2,The Grand Budapest Hotel,2014,Adventure,99,8.0716
3,Sherlock Jr,1924,Action,45,8.0704
4,Le salaire de la peur,1953,Adventure,131,8.0702
...,...,...,...,...,...
245,The Lives of Others,2006,Drama,137,8.3869
246,Sunset Blvd,1950,Drama,110,8.3838
247,Paths of Glory,1957,Drama,88,8.3712
248,The Shining,1980,Drama,146,8.3683


In [59]:
crew_cast

,title,director,cast,distributor
0,Harry Potter and the Deathly Hallows: Part 2,David Yates,"Daniel Radcliffe, Emma Watson",Warner Bros.
1,Mr Smith Goes to Washington,Frank Capra,"James Stewart, Jean Arthur",Columbia Pictures
2,The Grand Budapest Hotel,Wes Anderson,"Ralph Fiennes, F. Murray Abraham",Fox Searchlight Pictures
3,Sherlock Jr,Buster Keaton,"Buster Keaton, Kathryn McGuire",0
4,Le salaire de la peur,Henri-Georges Clouzot,"Yves Montand, Charles Vanel",0
...,...,...,...,...
245,The Lives of Others,Florian Henckel von Donnersmarck,"Ulrich Mühe, Martina Gedeck",Sony Pictures Classics
246,Sunset Blvd,Billy Wilder,"William Holden, Gloria Swanson",Paramount Pictures
247,Paths of Glory,Stanley Kubrick,"Kirk Douglas, Ralph Meeker",United Artists
248,The Shining,Stanley Kubrick,"Jack Nicholson, Shelley Duvall",Warner Bros.


In [60]:
income

,title,domestic_k,international_k,worldwide_k,dom_pct,int_pct
0,Harry Potter and the Deathly Hallows: Part 2,381447.587,960912.354,1342359.942,0.2842,0.7158
1,Mr Smith Goes to Washington,144.738,0.000,144.738,1.0000,0.0000
2,The Grand Budapest Hotel,59301.324,113780.865,173082.189,0.3426,0.6574
3,Sherlock Jr,0.000,0.000,0.000,0.0000,0.0000
4,Le salaire de la peur,0.000,1.098,1.098,0.0000,1.0000
...,...,...,...,...,...,...
245,The Lives of Others,11286.112,66070.830,77356.942,0.1459,0.8541
246,Sunset Blvd,299.645,0.428,300.073,0.9986,0.0014
247,Paths of Glory,0.000,5.252,5.252,0.0000,1.0000
248,The Shining,45634.352,1665.108,47299.460,0.9648,0.0352


In [61]:
ranking

,title,place,duration_rank,income_rank,rank_diff
0,Harry Potter and the Deathly Hallows: Part 2,187,107.5,4.0,-183.0
1,Mr Smith Goes to Washington,188,115.0,216.0,28.0
2,The Grand Budapest Hotel,189,211.5,87.0,-102.0
3,Sherlock Jr,190,250.0,245.5,55.5
4,Le salaire de la peur,191,101.5,239.0,48.0
...,...,...,...,...,...
245,The Lives of Others,58,82.5,114.0,56.0
246,Sunset Blvd,59,182.5,210.0,151.0
247,Paths of Glory,60,239.0,238.0,178.0
248,The Shining,61,62.5,132.0,71.0


In [62]:
# Genre tables
drama = pd.DataFrame(df_film[df_film['genre'] == 'Drama'][['title', 'place_bygenre', 'duration_rank_bygenre', 'income_rank_bygenre']].reset_index(drop = True))
make_checkpoint()
action = pd.DataFrame(df_film[df_film['genre'] == 'Action'][['title', 'place_bygenre', 'duration_rank_bygenre', 'income_rank_bygenre']].reset_index(drop = True))
make_checkpoint()
adventure = pd.DataFrame(df_film[df_film['genre'] == 'Adventure'][['title', 'place_bygenre', 'duration_rank_bygenre', 'income_rank_bygenre']].reset_index(drop = True))
make_checkpoint()
crime = pd.DataFrame(df_film[df_film['genre'] == 'Crime'][['title', 'place_bygenre', 'duration_rank_bygenre', 'income_rank_bygenre']].reset_index(drop = True))
make_checkpoint()
comedy = pd.DataFrame(df_film[df_film['genre'] == 'Comedy'][['title', 'place_bygenre', 'duration_rank_bygenre', 'income_rank_bygenre']].reset_index(drop = True))
make_checkpoint()
biography = pd.DataFrame(df_film[df_film['genre'] == 'Biography'][['title', 'place_bygenre', 'duration_rank_bygenre', 'income_rank_bygenre']].reset_index(drop = True))
make_checkpoint()

In [63]:
Animation = pd.DataFrame(df_film[df_film['genre'] == 'Animation'][['title', 'place_bygenre', 'duration_rank_bygenre', 'income_rank_bygenre']].reset_index(drop = True))
Horror = pd.DataFrame(df_film[df_film['genre'] == 'Horror'][['title', 'place_bygenre', 'duration_rank_bygenre', 'income_rank_bygenre']].reset_index(drop = True))
Mystery = pd.DataFrame(df_film[df_film['genre'] == 'Mystery'][['title', 'place_bygenre', 'duration_rank_bygenre', 'income_rank_bygenre']].reset_index(drop = True))
Western = pd.DataFrame(df_film[df_film['genre'] == 'Western'][['title', 'place_bygenre', 'duration_rank_bygenre', 'income_rank_bygenre']].reset_index(drop = True))
Film_Noir = pd.DataFrame(df_film[df_film['genre'] == 'Film-Noir'][['title', 'place_bygenre', 'duration_rank_bygenre', 'income_rank_bygenre']].reset_index(drop = True))
frames = [Animation, Horror, Mystery, Western, Film_Noir]
other_genre = pd.concat(frames).reset_index(drop = True)
make_checkpoint()

In [64]:
drama

,title,place_bygenre,duration_rank_bygenre,income_rank_bygenre
0,Room,17.0,47.5,29.0
1,Det sjunde inseglet,16.0,61.0,58.0
2,Pather Panchali,15.0,38.0,60.0
3,Tôkyô monogatari,14.0,25.0,64.0
4,Platoon,13.0,43.5,13.0
...,...,...,...,...
63,Django Unchained,56.0,6.5,4.0
64,The Lives of Others,55.0,24.0,20.0
65,Sunset Blvd,54.0,53.0,57.0
66,Paths of Glory,53.0,66.0,67.0


In [65]:
action

,title,place_bygenre,duration_rank_bygenre,income_rank_bygenre
0,Sherlock Jr,10.0,48.0,47.0
1,The Batman,9.0,6.0,12.0
2,How to Train Your Dragon,8.0,46.0,18.0
3,Mad Max: Fury Road,7.0,34.5,25.0
4,Ford v Ferrari,6.0,12.5,29.0
5,The Terminator,5.0,44.5,37.0
6,Rush,4.0,32.0,36.0
7,Logan,3.0,21.0,15.0
8,The Incredibles,2.0,39.5,14.0
9,Pirates of the Caribbean: The Curse of the Bla...,1.0,17.0,13.0


In [66]:
adventure

,title,place_bygenre,duration_rank_bygenre,income_rank_bygenre
0,Harry Potter and the Deathly Hallows: Part 2,14.0,15.0,1.0
1,The Grand Budapest Hotel,13.0,26.0,20.0
2,Le salaire de la peur,12.0,14.0,36.0
3,Barry Lyndon,11.0,3.0,32.0
4,Klaus,10.0,28.5,37.0
5,Jaws,9.0,18.0,13.0
6,"Monsters, Inc",8.0,32.0,10.0
7,Stand by Me,7.0,35.0,24.0
8,Into the Wild,6.0,11.0,23.0
9,The Wizard of Oz,5.0,24.0,27.0


In [67]:
crime

,title,place_bygenre,duration_rank_bygenre,income_rank_bygenre
0,Salinui chueok,5.0,13.0,21.0
1,Cool Hand Luke,4.0,16.5,33.5
2,Les quatre cents coups,3.0,29.5,25.0
3,La haine,2.0,31.5,24.0
4,Du rififi chez les hommes,1.0,20.5,23.0
5,Witness for the Prosecution,24.0,23.0,31.0
6,Joker,23.0,18.5,1.0
7,Once Upon a Time in America,22.0,1.0,19.0
8,Reservoir Dogs,21.0,29.5,20.0
9,Tengoku to jigoku,20.0,10.0,26.5


In [68]:
comedy

,title,place_bygenre,duration_rank_bygenre,income_rank_bygenre
0,Mr Smith Goes to Washington,8.0,3.5,20.0
1,Relatos salvajes,7.0,8.5,12.0
2,The Big Lebowski,6.0,11.0,11.0
3,Dead Poets Society,5.0,5.0,3.0
4,Groundhog Day,4.0,18.0,9.0
5,To Be or Not to Be,3.0,19.0,23.5
6,Life of Brian,2.0,21.0,13.0
7,It Happened One Night,1.0,14.0,23.5
8,Dr Strangelove or: How I Learned to Stop Worry...,19.0,20.0,15.0
9,Singin' in the Rain,18.0,16.5,16.0


In [69]:
biography

,title,place_bygenre,duration_rank_bygenre,income_rank_bygenre
0,Hacksaw Ridge,7.0,12.0,9.0
1,La passion de Jeanne d'Arc,6.0,22.0,22.0
2,Hotel Rwanda,5.0,20.0,19.0
3,Spotlight,4.0,17.5,12.0
4,Hachi: A Dog's Tale,3.0,23.0,18.0
5,The Sound of Music,2.0,5.0,10.0
6,Gandhi,1.0,2.0,15.0
7,Braveheart,19.0,4.0,7.0
8,Amadeus,18.0,6.5,16.0
9,Hamilton,17.0,6.5,23.0


In [70]:
other_genre

,title,place_bygenre,duration_rank_bygenre,income_rank_bygenre
0,Mary and Max,2.0,2.0,4.0
1,Beauty and the Beast,1.0,5.0,1.0
2,Kimi no na wa,4.0,1.0,2.0
3,Tonari no Totoro,3.0,4.0,3.0
4,Hotaru no haka,5.0,3.0,5.0
5,The Exorcist,1.0,1.0,1.0
6,The Thing,2.0,3.5,4.0
7,Psycho,4.0,3.5,3.0
8,Alien,3.0,2.0,2.0
9,Vertigo,2.0,2.0,4.0


In [71]:
# Convert all tables into spark data frame
df_film_info_spark = spark.createDataFrame(film_info)
df_crew_cast_spark = spark.createDataFrame(crew_cast)
df_income_spark = spark.createDataFrame(income)
df_ranking_spark = spark.createDataFrame(ranking)
df_drama_spark = spark.createDataFrame(drama)
df_action_spark = spark.createDataFrame(action)
df_adventure_spark = spark.createDataFrame(adventure)
df_crime_spark = spark.createDataFrame(crime)
df_comedy_spark = spark.createDataFrame(comedy)
df_biography_spark = spark.createDataFrame(biography)
df_other_genre_spark = spark.createDataFrame(other_genre)

In [72]:
df_film_info_spark.printSchema()
df_crew_cast_spark.printSchema()
df_income_spark.printSchema()
df_ranking_spark.printSchema()
df_drama_spark.printSchema()
df_action_spark.printSchema()
df_adventure_spark.printSchema()
df_crime_spark.printSchema()
df_comedy_spark.printSchema()
df_biography_spark.printSchema()
df_other_genre_spark.printSchema()

root
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)
 |-- genre: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- rating: double (nullable = true)

root
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- distributor: string (nullable = true)

root
 |-- title: string (nullable = true)
 |-- domestic_k: double (nullable = true)
 |-- international_k: double (nullable = true)
 |-- worldwide_k: double (nullable = true)
 |-- dom_pct: double (nullable = true)
 |-- int_pct: double (nullable = true)

root
 |-- title: string (nullable = true)
 |-- place: long (nullable = true)
 |-- duration_rank: double (nullable = true)
 |-- income_rank: double (nullable = true)
 |-- rank_diff: double (nullable = true)

root
 |-- title: string (nullable = true)
 |-- place_bygenre: double (nullable = true)
 |-- duration_rank_bygenre: double (nullable = true)
 |-- income_rank_bygenre: double (nullable = tru

In [73]:
# Convert all data frames into parquet files
df_film_info_spark.write.parquet("/project/Individual/parquet_files/film_info.parquet", mode = 'overwrite')
df_crew_cast_spark.write.parquet("/project/Individual/parquet_files/crew_cast.parquet", mode = 'overwrite')
df_income_spark.write.parquet("/project/Individual/parquet_files/income.parquet", mode = 'overwrite')
df_ranking_spark.write.parquet("/project/Individual/parquet_files/ranking.parquet", mode = 'overwrite')
df_drama_spark.write.parquet("/project/Individual/parquet_files/drama.parquet", mode = 'overwrite')
df_action_spark.write.parquet("/project/Individual/parquet_files/action.parquet", mode = 'overwrite')
df_adventure_spark.write.parquet("/project/Individual/parquet_files/adventure.parquet", mode = 'overwrite')
df_crime_spark.write.parquet("/project/Individual/parquet_files/crime.parquet", mode = 'overwrite')
df_comedy_spark.write.parquet("/project/Individual/parquet_files/comedy.parquet", mode = 'overwrite')
df_biography_spark.write.parquet("/project/Individual/parquet_files/biography.parquet", mode = 'overwrite')
df_other_genre_spark.write.parquet("/project/Individual/parquet_files/other_genre.parquet", mode = 'overwrite')
make_checkpoint()